# 3. Using Included ProgPy Models

ProgPy is distributed with a few pre-constructed models that can be used in simulation or prognostics. These models for batteries, pumps, valves, among others, are included in the `progpy.models` package.

In this notebook, we will be exploring a generalized overview of each included model. For more in-depth descriptions of the included models, please refer to the [Included Models](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html) documentation.

## Table of Contents
* [Battery Models](#Battery-Models)
    * [Battery Circuit](#BatteryCircuit)
    * [BatteryElectroChemEOD](#BatteryElectroChemEOD)
    * [BatteryElectroChemEOL](#BatteryElectroChemEOL)
    * [Combined BatteryElectroChem (BatteryElectroChemEODEOL)](#Combined-BatteryElectroChem-(BatteryElectroChemEODEOL))

## Battery Models

In the following battery models, the default model parameters included are for Li-ion batteries, specifically 18650-type cells. Experimental discharge curves for these cells can be downloaded from the Prognostics Center of Excellence [Data Repository](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository).

### BatteryCircuit

In this first example, we will demonstrate how to set up, configure, and use the BatteryCiruit model. The BatteryCircuit model is a vectorized prognostics model for a battery, represented by an equivalent circuit model as described in [[Daigle Sankararaman 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2253).

We will start by importing the model and initializing a battery instance with default settings.

In [ ]:
from progpy.models import BatteryCircuit

batt = BatteryCircuit()

Information is passed to and from the model using containers that function like dictionaries. The keys of the containers are specific to the model. Let's look at the inputs (loading) and outputs (measurements) for the BatteryCircuit model.

In [ ]:
print('inputs:', batt.inputs)
print('outputs:', batt.outputs)

If we refer to the [documentation](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html), we can see that the input `i` refers to the current draw on the battery. The outputs `t` refers to the temperature in units Kelvin and `v` refers to voltage.

We can also print out what events we're predicting and the internal states the model uses to represent the system.

In [ ]:
print('event(s): ', batt.events)
print('states: ', batt.states)

We can see that this particular model only predicts one event, called `EOD` (End of Discharge). The states listed include `tb`, the battery temperature in K; `qb`, the charge stored in Capacitor Cb of the equivalent circuit model; `qcp`, the charge stored in Capacitor Ccp of the equivalent circuit model; and `qcs`, the charge stored in Capacitor Ccs of the equivalent circuit model.

Let's now look at the model's configuration parameters, which describe the specific system (in this case, the battery) that the model is simulating.

In [ ]:
from pprint import pprint

print('Model configuration:')
pprint(batt.parameters)

We can set the configuration by modifying parameters while initializing the battery or after. The BatteryCircuit default parameters are for a 18650 battery tested in NASA's SHARP lab. If you're using the model for a system other than that one battery, you will need to update the parameters.

In [ ]:
batt.parameters['qMax'] = 7856 
batt.parameters['process_noise'] = 0  # Turns off default process noise

We can save or load a model configuration using pickle.

In [ ]:
import pickle
pickle.dump(batt.parameters, open('battery123.cfg', 'wb'))

We can then set your model configuration like below. This is useful for recording and restoring model configurations. Some users store model configuration as picked files to be restored from later.

In [ ]:
batt.parameters = pickle.load(open('battery123.cfg', 'rb'))

Additionally, both pickle and JSON can be used to serialize an entire model for future use. All models include functions to serialize with JSON.

In [ ]:
save_json = batt.to_json()
serial_batt = BatteryCircuit.from_json(save_json)

Let's now use the model to do a simulation. To do this, we will first need to set a configuration and define a future load. For more details on future loading, refer to the related section in __[01 Simulation](01_Simulation.ipynb#Future-Loading)__.

In [ ]:
config = {
    'save_freq': 100,
    'dt': 2,
    't0': 700,
}

def future_loading(t, x=None):
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2     
    else:
        i = 3
    return batt.InputContainer({'i': i})

Let's run the simulation and plot the inputs and outputs. We can do this using the built-in [plot method](https://nasa.github.io/progpy/api_ref/progpy/SimResult.html#progpy.sim_result.SimResult.plot) based on matplotlib or with other imported plotting libraries.

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, **config)

fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current draw (amps)')

In [ ]:
def print_battery_output_plots(results):
    fig = results.outputs.plot(keys=['t'], xlabel='time (s)', ylabel='temperature (K)', figsize=(10,4))
    fig2 = results.outputs.plot(keys=['v'], xlabel='time (s)', ylabel='voltage (V)', figsize=(10,4))

print_battery_output_plots(simulated_results)

### BatteryElectroChemEOD

BatteryElectroChemEOD is a vectorized prognostics model for a battery, represented by an electrochemical equations as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252). This model predicts the end of discharge event.

In [ ]:
from progpy.models import BatteryElectroChemEOD

batt = BatteryElectroChemEOD()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('outputs:', batt.states)

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, **config)

fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current draw (amps)')

In [ ]:
print_battery_output_plots(simulated_results)

### BatteryElectroChemEOL

BatteryElectroChemEOL is a vectorized prognostics model for battery degredation, represented by an electrochemical model as described in [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132)

In [ ]:
from progpy.models import BatteryElectroChemEOL

batt = BatteryElectroChemEOL()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('outputs:', batt.states)

### Combined BatteryElectroChem (BatteryElectroChemEODEOL)

BatteryElectroChemEODEOL is a prognostics model for battery degredation and discharge, represented by an electrochemical model as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252) and [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132).

In [ ]:
from progpy.models import BatteryElectroChem

batt = BatteryElectroChem()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('outputs:', batt.states)

## Centrifugal Pump Model

## Electric Powertrain Models

## Pneumatic Valve Model

## Aircraft Flight Model